In [2]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import json

# Load the .mat file
groundTruth_data = scipy.io.loadmat('groundTruth_Cuprite_nEnd12.mat')
Cuprite_data = scipy.io.loadmat('Cuprite.mat')



In [3]:
Cuprite=Cuprite_data['X'] 
print(Cuprite.shape)
  
reference_spectra = np.transpose(groundTruth_data['M'])
print(reference_spectra.shape)
 

(512, 614, 224)
(12, 224)


In [4]:
max_val = np.max(reference_spectra)
min_val = np.min(reference_spectra)

print("Maximum value of reference_spectra:", max_val)
print("Minimum value of reference_spectra:", min_val)

max_val = np.max(Cuprite)
min_val = np.min(Cuprite)

print("Maximum value of Cuprite:", max_val)
print("Minimum value of Cuprite:", min_val)

Maximum value of reference_spectra: 0.9120257608554999
Minimum value of reference_spectra: 0.0770244938180001
Maximum value of Cuprite: 12000
Minimum value of Cuprite: -50


In [5]:
Cuprite_shifted = Cuprite + 50 
Cuprite_norm = Cuprite_shifted / 12150 
Cuprite_norm = Cuprite_norm + 1e-6 
print("Minimum value in normalized array:", np.min(Cuprite_norm))
print("Maximum value in normalized array:", np.max(Cuprite_norm))

Minimum value in normalized array: 1e-06
Maximum value in normalized array: 0.991770547325103


In [6]:
def avg_threshold(image, ref_spectrum):
    dot_products = np.sum(image * ref_spectrum, axis=2) 
    norms_image = np.sqrt(np.sum(np.square(image), axis=2))
    norm_ref_spectrum = np.linalg.norm(ref_spectrum) 
    cos_sims = dot_products / (norms_image * norm_ref_spectrum) 
    avg_cos_sim = np.mean(cos_sims)

    return avg_cos_sim


In [7]:
def count_similar_pixels(image, ref_spectrum, threshold):
    # compute dot products between reference spectrum and all pixels in image
    dot_products = np.dot(image.reshape(-1, image.shape[-1]), ref_spectrum)
    
    # reshape dot products to have same shape as image
    cos_sims = dot_products.reshape(image.shape[:-1])
    
    # compute norms of image pixels and reference spectrum
    norms_image = np.sqrt(np.sum(np.square(image), axis=2))
    norm_ref_spectrum = np.sqrt(np.sum(np.square(ref_spectrum)))
    
    # compute cosine similarities between reference spectrum and all pixels in image
    cos_sims /= (norms_image * norm_ref_spectrum)
    
    # count number of pixels with cosine similarity greater than threshold
    num_similar_pixels = np.sum(cos_sims > threshold)
    
    return num_similar_pixels

In [8]:
data={
  "Alunite": {
    "Threshold": 0.930899999,
    "No. of data points": 707
  },
  "Andradite": {
    "Threshold": 0.9466,
    "No. of data points": 1039
  },
  "Buddingtonite": {
    "Threshold": 0.94101,
    "No. of data points": 920
  },
  "Dumortierite": {
    "Threshold": 0.93900,
    "No. of data points": 989
  },
  "Kaolinite1": {
    "Threshold": 0.9488,
    "No. of data points": 327
  },
  "Kaolinite2": {
    "Threshold": 0.9550989,
    "No. of data points": 1482
  },
  "Muscovite": {
    "Threshold": 0.9430000,
    "No. of data points": 1259
  },
  "Montmorillonite": {
    "Threshold": 0.9523,
    "No. of data points": 1677
  },
  "Nontronite": {
    "Threshold": 0.9615,
    "No. of data points": 1066
  },
  "Pyrope": {
    "Threshold": 0.949222,
    "No. of data points": 474
  },
  "Sphene": {
    "Threshold": 0.9440,
    "No. of data points": 343
  },
  "Chalcedony": {
    "Threshold": 0.94658999,
    "No. of data points": 1079
  }
}


In [18]:
elem="Kaolinite1"
ref_spectrum = reference_spectra[4] 
image=Cuprite_norm

In [20]:
threshold_value =0.9488
num_similar_pixels = count_similar_pixels(image, ref_spectrum, threshold_value)
print("Number of data points with cosine similarity > {}: {}".format(threshold_value, num_similar_pixels))

Number of data points with cosine similarity > 0.9488: 327


In [11]:
data[elem]['threshold'] = threshold_value
data[elem]['no_of_data_points'] = num_similar_pixels

In [12]:
for element, info in data.items():
    print(f"{element}: Threshold = {info['Threshold']}, No. of data points = {info['No. of data points']}")


Alunite: Threshold = 0.930899999, No. of data points = 707
Andradite: Threshold = 0.9466, No. of data points = 1039
Buddingtonite: Threshold = 0.94101, No. of data points = 920
Dumortierite: Threshold = 0.939, No. of data points = 989
Kaolinite1: Threshold = 0.9482, No. of data points = 534
Kaolinite2: Threshold = 0.9555989, No. of data points = 1034
Muscovite: Threshold = 0.9435, No. of data points = 953
Montmorillonite: Threshold = 0.9529, No. of data points = 1062
Nontronite: Threshold = 0.9619, No. of data points = 739
Pyrope: Threshold = 0.949222, No. of data points = 474
Sphene: Threshold = 0.943523, No. of data points = 534
Chalcedony: Threshold = 0.95018999, No. of data points = 123


In [13]:
import pickle
 
with open('data.pkl', 'wb') as f:
    pickle.dump(data, f)


In [14]:
import pickle
with open('data.pkl', 'rb') as f:
    data = pickle.load(f)
print(data)

{'Alunite': {'Threshold': 0.930899999, 'No. of data points': 707}, 'Andradite': {'Threshold': 0.9466, 'No. of data points': 1039}, 'Buddingtonite': {'Threshold': 0.94101, 'No. of data points': 920}, 'Dumortierite': {'Threshold': 0.939, 'No. of data points': 989}, 'Kaolinite1': {'Threshold': 0.9482, 'No. of data points': 534}, 'Kaolinite2': {'Threshold': 0.9555989, 'No. of data points': 1034}, 'Muscovite': {'Threshold': 0.9435, 'No. of data points': 953}, 'Montmorillonite': {'Threshold': 0.9529, 'No. of data points': 1062}, 'Nontronite': {'Threshold': 0.9619, 'No. of data points': 739}, 'Pyrope': {'Threshold': 0.949222, 'No. of data points': 474}, 'Sphene': {'Threshold': 0.943523, 'No. of data points': 534, 'threshold': 0.943523, 'no_of_data_points': 534}, 'Chalcedony': {'Threshold': 0.95018999, 'No. of data points': 123}}


# Assuming the dictionary is stored in a variable called 'data'
total_data_points = 0
for element in data.values():
    total_data_points += data['no_of_data_points']

print("Total number of data points:", total_data_points)
